# DREAM powder data reduction

- Audience: Instrument users, beginners
- Prerequisites: Basic knowledge of [Scipp](https://scipp.github.io/)

This notebook demonstrates the basic powder data reduction workflow for DREAM in high-flux mode.
The workflow
1. converts the data to $d$-spacing,
2. focuses into a single bin per $d$-spacing (i.e., removes dependency on voxel position),
2. normalizes by a monitor histogram,
3. normalizes by a vanadium measurement,
4. and saves the results to a time-of-flight CIF file for use by [EasyDiffraction](https://easydiffraction.org/).

Only the mantle detector of DREAM is used in this workflow for simplicity.
See [DREAM advanced powder data reduction](./dream-advanced-powder-reduction.rst) for more options.

This notebook uses data from a McStas + GEANT4 simulation but can be adapted to other data.
The data is available through the ESSdiffraction package but accessing it requires the `pooch` package.
If you get an error about a missing module `pooch`, you can install it with `!pip install pooch`:

In [ ]:
import scipp as sc
from ess import dream, powder
import ess.dream.data  # noqa: F401
from ess.powder.types import *

## Create and configure the workflow

We begin by creating the Dream (Geant4) workflow object which is a skeleton for reducing Dream data, with pre-configured steps.

In [ ]:
workflow = dream.DreamGeant4Workflow(run_norm=powder.RunNormalization.monitor_histogram)

We then need to set the missing parameters which are specific to each experiment
(the keys are types defined in [ess.powder.types](../../generated/modules/ess.powder.types.rst)).

In [ ]:
workflow[Filename[SampleRun]] = dream.data.simulated_diamond_sample()
workflow[Filename[VanadiumRun]] = dream.data.simulated_vanadium_sample()
workflow[Filename[BackgroundRun]] = dream.data.simulated_empty_can()
workflow[CalibrationFilename] = None

workflow[MonitorFilename[SampleRun]] = dream.data.simulated_monitor_diamond_sample()
workflow[MonitorFilename[VanadiumRun]] = dream.data.simulated_monitor_vanadium_sample()
workflow[MonitorFilename[BackgroundRun]] = dream.data.simulated_monitor_empty_can()
workflow[CaveMonitorPosition] = sc.vector([0.0, 0.0, -4220.0], unit="mm")

workflow[dream.InstrumentConfiguration] = dream.InstrumentConfiguration.high_flux
# Select a detector bank:
workflow[NeXusDetectorName] = "mantle"
# We drop uncertainties where they would otherwise lead to correlations:
workflow[UncertaintyBroadcastMode] = UncertaintyBroadcastMode.drop
# Edges for binning in d-spacing:
workflow[DspacingBins] = sc.linspace("dspacing", 0.3, 2.3434, 201, unit="angstrom")

# Do not mask any pixels / voxels:
workflow = powder.with_pixel_mask_filenames(workflow, [])

## Use the reduction workflow

We then call [workflow.compute(target)](https://scipp.github.io/sciline/generated/classes/sciline.Pipeline.html#sciline.Pipeline.compute) to compute the result.
`IofTof` and `ReducedTofCIR` encode the same data but we can use the former to make plots and the latter to write a CIF file.

In [ ]:
results = workflow.compute([IofTof, ReducedTofCIF])
result = results[IofTof]
cif_data = results[ReducedTofCIF]

In [ ]:
histogram = result.hist()
histogram.plot()

We can now save the result to disk:

In [ ]:
cif_data.comment = """This file was generated with the basic DREAM data reduction user guide
in the documentation of ESSdiffraction.
See https://scipp.github.io/essdiffraction/
"""
cif_data.save("reduced.cif")

## Visualize the workflow

We can visualize the workflow as a graph.
This can help us understand how the data was reduced.

In [ ]:
workflow.visualize(ReducedTofCIF, graph_attr={"rankdir": "LR"})

The workflow is a [Sciline](https://scipp.github.io/sciline/) pipeline.
See the documentation of Sciline for more information and how to modify and extend the workflow.

Next, consider reading [DREAM advanced powder data reduction](./dream-advanced-powder-reduction.rst) guide which demonstrates some builtin options for customizing the workflow and how to compute alternative results.